In [37]:
# Imports
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import itertools
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import pickle

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Modeling

In this notebook, we are taking the output from 3_EDA to start doing the modeling. To recap on the missing steps, we will:

#### `3. Model Selection`
Given the nature the data, we might consider the following models:
- **ARIMA/SARIMA**: If your data shows trends or autocorrelation. SARIMA is suitable if there is a clear seasonal pattern.
- **Prophet**: Handles daily data well, robust to missing data, and good with seasonality.
- **Machine Learning Approaches**: If there are other factors that can predict 'quantity', a model like Random Forest or Gradient Boosting might be useful.

#### 4. Model Training and Forecasting
- **Training**: Training the yearly data.
- **Forecasting**: Generate predictions for the upcomming days.
- **Hyperparameter Tuning**: Optimize model parameters for best performance.

#### 5. Model Evaluation
- **Performance Metrics**: Evaluate the model using appropriate metrics.
- **Cross-Validation**: If possible, use time series cross-validation.

Here's a general approach to fitting an ARIMA model:

Selecting Model Orders: You need to choose the SARIMA model orders. These include the non-seasonal parameters (p, d, q) and the seasonal. Since you've detrended the data, d might be 0 or 1 depending on if the data needs further differencing to achieve stationarity. The seasonal parameters will incorporate the seasonality you've observed.

Seasonal Period (s): This is the seasonality of the data. In your case, if the seasonality is twice per month, you've already calculated the seasonal period to be approximately 365 hours.

Parameter Estimation: Use the ACF and PACF plots of the detrended data to estimate the initial ARIMA parameters. You can also use grid search methods to test different combinations of parameters and select the best model based on some criteria like the AIC (Akaike Information Criterion).

Model Diagnostics: After fitting the model, check the diagnostics to ensure that the residuals of your model are white noise (no autocorrelation).

# Modeling by Country

We will do a separate model for each country taking the signla processed from the last EDA step.

In [6]:
# Parsing date strings, ignoring any timezone information and converting them to datetime objects
date_parser = lambda x: pd.to_datetime(x[:22])

## SPAIN Model

In [19]:
# Spain model
model_spain_df = pd.read_csv("../3_EDA/EDA_countries/EDA_SP.csv", 
                     converters={'EndTime': date_parser}).set_index('EndTime').dropna()

Initial Model

In [29]:
warnings.filterwarnings("ignore")

p = 1  # replace with actual
d = 1  # replace with actual
q = 1  # replace with actual

# Fit the ARIMA model (using the known part of the time series)
model = ARIMA(model_spain_df['detrended'], order=(p, d, q))
model_fit = model.fit()

# Forecast the missing values
# The 'steps' argument would be the number of hours in the missing months
forecast = model_fit.get_forecast(steps=1000) #hours in a monyh

# The forecast object contains the predicted values and other information
forecasted_values = forecast.predicted_mean

# You may also want to extract the confidence intervals of the forecasts
conf_int = forecast.conf_int()

Hyperparameter Tunning

In [22]:
warnings.filterwarnings("ignore")

# Assuming model_spain_df is your DataFrame with 'surplus' as the column

# Define the p, d, and q ranges
p = d = q = range(0, 3)  # Example ranges; you can adjust these

# Generate all different combinations of p, d, and q triplets
pdq = list(itertools.product(p, d, q))

best_aic = float("inf")
best_order = None
best_model = None

for order in pdq:
    try:
        model = ARIMA(model_spain_df['detrended'], order=order)
        model_fit = model.fit()
        if model_fit.aic < best_aic:
            best_aic = model_fit.aic
            best_order = order
            best_model = model_fit
    except:
        continue

print(f'Best ARIMA{best_order} AIC: {best_aic}')

# Use best_model for forecasts
forecast = best_model.get_forecast(steps=1)  # Adjust 'steps' as needed
forecasted_values = forecast.predicted_mean
conf_int = forecast.conf_int()


Best ARIMA(2, 1, 2) AIC: 155864.65620640424


In [38]:
# Open a file in write binary mode
with open('../models/best_arima_model_SP.pkl', 'wb') as file:
    pickle.dump(best_model, file)

## POLAND Model

## DENMARK Model

## SWEEDEN Model

## NETHERLANDS Model

## GERMANY Model

## UK Model

## ITALIA Model

## HUNGARY Model